In [8]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._

import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._


In [9]:
val inputStreamPath = "./data/events-stream"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath: String = ./data/events-stream
dataSchema: org.apache.spark.sql.types.StructType = StructType(StructField(tweet,StringType,true))
inputDF: org.apache.spark.sql.DataFrame = [tweet: string]


In [10]:
val trainedModel = PipelineModel.load("./models/spark-ml-model-rf")
val getProbabilityNegativeTweet = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))
val getProbabilityPositiveTweet = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(0))

trainedModel: org.apache.spark.ml.PipelineModel = pipeline_17a6fd2fa90f
getProbabilityNegativeTweet: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))
getProbabilityPositiveTweet: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


In [11]:
val predictionDf = trainedModel.transform(inputDF)

predictionDf.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF
        .withColumn("probability_negative", getProbabilityNegativeTweet($"probability"))
        .withColumn("probability_positive", getProbabilityPositiveTweet($"probability"))
        .withColumn("is_positive", when($"probability_positive" > 0.5, 1).otherwise(0))
    .show()
}.start()

predictionDf: org.apache.spark.sql.DataFrame = [tweet: string, words: array<string> ... 6 more fields]
res4: org.apache.spark.sql.streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@280788ff


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_negative|probability_positive|is_positive|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|wishes everyone d...|[wishes, everyone...|(1000,[77,388,748...|(1000,[77,388,748...|[4.89510838998072...|[0.48951083899807...|       1.0|             1|   0.510489161001927|   0.489510838998073|          0|
|Can't sleep... I ...|[can't, sleep...,...|(1000,[82,225,233...|(1000,[82,225,233...|[5.24037086071899...|[0.52403708607189...|       0.0|             0| 0.475962913928


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_negative|probability_positive|is_positive|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|Good Morning.. we...|[good, morning..,...|(1000,[37,57,77,1...|(1000,[37,57,77,1...|[5.50444084723814...|[0.55044408472381...|       0.0|             0|  0.4495559152761855|  0.5504440847238146|          1|
|ew i hate math so...|[ew, i, hate, mat...|(1000,[41,69,77,3...|(1000,[41,69,77,3...|[5.19415362052534...|[0.51941536205253...|       0.0|             0| 0.48058463794

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_negative|probability_positive|is_positive|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|Trying to fall as...|[trying, to, fall...|(1000,[18,134,213...|(1000,[18,134,213...|[4.89510838998072...|[0.48951083899807...|       1.0|             1|   0.510489161001927|   0.489510838998073|          0|
|@veshiy0leg  NO b...|[@veshiy0leg, , n...|(1000,[37,149,260...|(1000,[37,149,260...|[5.31693955087659...|[0.53169395508765...|       0.0|             0|  0.46830604491


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_negative|probability_positive|is_positive|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|Time to don my ga...|[time, to, don, m...|(1000,[157,204,33...|(1000,[157,204,33...|[5.50444084723814...|[0.55044408472381...|       0.0|             0|  0.4495559152761855|  0.5504440847238146|          1|
|@commensense It i...|[@commensense, it...|(1000,[25,36,83,1...|(1000,[25,36,83,1...|[5.38613306051866...|[0.53861330605186...|       0.0|             0| 0.46138669394

...


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_negative|probability_positive|is_positive|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|finished watching...|[finished, watchi...|(1000,[10,79,123,...|(1000,[10,79,123,...|[4.89510838998072...|[0.48951083899807...|       1.0|             1|   0.510489161001927|   0.489510838998073|          0|
|@Keiyaunna So i g...|[@keiyaunna, so, ...|(1000,[14,329,368...|(1000,[14,329,368...|[5.06101264098972...|[0.50610126409897...|       0.0|             0| 0.49389873590


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_negative|probability_positive|is_positive|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|@oohsoofreshmo I ...|[@oohsoofreshmo, ...|(1000,[83,106,172...|(1000,[83,106,172...|[4.90526580352168...|[0.49052658035216...|       1.0|             1|  0.5094734196478318|  0.4905265803521681|          0|
|let today pass fa...|[let, today, pass...|(1000,[100,164,17...|(1000,[100,164,17...|[4.78894772834674...|[0.47889477283467...|       1.0|             1|  0.5211052271


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_negative|probability_positive|is_positive|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|Feeling very disc...|[feeling, very, d...|(1000,[65,162,944...|(1000,[65,162,944...|[5.07851230451630...|[0.50785123045163...|       0.0|             0|  0.4921487695483693|  0.5078512304516307|          1|
|@MikeShanePhoto n...|[@mikeshanephoto,...|(1000,[18,130,842...|(1000,[18,130,842...|[4.89510838998072...|[0.48951083899807...|       1.0|             1|   0.510489161


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_negative|probability_positive|is_positive|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|I don't like good...|[i, don't, like, ...|(1000,[329,330,42...|(1000,[329,330,42...|[4.89510838998072...|[0.48951083899807...|       1.0|             1|   0.510489161001927|   0.489510838998073|          0|
|@Kates06 grr wat ...|[@kates06, grr, w...|(1000,[19,36,48,1...|(1000,[19,36,48,1...|[4.89510838998072...|[0.48951083899807...|       1.0|             1|   0.510489161


+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|probability_negative|probability_positive|is_positive|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+--------------------+--------------------+-----------+
|I must to say tha...|[i, must, to, say...|(1000,[19,23,34,7...|(1000,[19,23,34,7...|[4.78894772834674...|[0.47889477283467...|       1.0|             1|  0.5211052271653251| 0.47889477283467485|          0|
|New blog post . S...|[new, blog, post,...|(1000,[25,56,145,...|(1000,[25,56,145,...|[5.24779347763553...|[0.52477934776355...|       0.0|             0|  0.4752206522